# Setup

## Environment

1. Create a virtual environment or similar (this was built with Python 3.10, but 3.11 should work too), and install `requirements.txt`:
    ```bash
    pip install -r requirements.txt
    ```
2. Setup Google Cloud Application Default Credentials (see [this doc](https://cloud.google.com/docs/authentication/provide-credentials-adc)).
3. Copy the `.env.template` file and set keys and other information as indicated.

## Connection Setup

Load the `.env` file into the Python environment:

In [1]:
from dotenv import load_dotenv
load_dotenv(override=True)  

True

Initialize VertexAI:

In [2]:
import os
import vertexai
vertexai.init(
    project=os.environ.get("GOOGLE_PROJECT_NAME"),
    location=os.environ.get("GOOGLE_LOCATION",'us-east1'),
)

Initialize LangChain VertexAI components:

In [18]:
from langchain_google_vertexai import VertexAI
from langchain_google_vertexai import ChatVertexAI
from langchain_google_vertexai import VertexAIEmbeddings
from vertexai.generative_models import GenerativeModel

llmModel = VertexAI(model_name=os.environ.get('GOOGLE_LLM','gemini-1.5-flash'))
chatModel = ChatVertexAI(model=os.environ.get('GOOGLE_LLM','gemini-1.5-flash'), temperature=0)
embedModel = VertexAIEmbeddings(model_name=os.environ.get('GOOGLE_EMBED_MODEL','multimodalembedding')) 
genModel = GenerativeModel(model_name=os.environ.get('GOOGLE_LLM','gemini-1.5-flash'))

Initialize Cassio (Astra DB)

In [4]:
import cassio
cassio.init(auto=True)

And establish the graph store:

In [5]:
from ragstack_langchain.graph_store import CassandraGraphStore     

SITE_PREFIX="travel_docs"
graph_store = CassandraGraphStore(
    embedModel,
    node_table=f"{SITE_PREFIX}_nodes",
    edge_table=f"{SITE_PREFIX}_edges")

# Create and Load LangChain `Document`s

The example `Tourbook.pdf` is fairly complex in structure, both digitally and visually. 

## Text

A variety of parsing tools such as Unstructured and Adobe ExtractAPI were attempted on `Tourbook.pdf` file, attempting with both file structure and OCR techniques, to no avail. The Vertex LLM was able to parse (with a fairly generic prompt), but unfortunately exited early as it determined it was repeating existing content and cited the URL of this!

In this notebook we are trying to demonstrate multi-modal embedding and retrieval, so the information was manually parsed, and put into the file `Tourbook.json`. This contains the first 24 pages minus the cover page, the table of contents, and a map on page 3.

Each paragraph will be linked to its closest (lowest) heading with a bi-directional link; this results in a graph linked through the headings.

In [6]:
from langchain_core.documents import Document
from ragstack_knowledge_store.link_tag import BidirLinkTag
import json
from hashlib import sha256

with open('Tourbook.json', 'r', encoding='utf-8') as file:
    text_data = json.load(file)

text_documents = []
h1_dict = {}

for i, entry in enumerate(text_data):
    # Capture the H1 header for each page for future reference
    h1 = entry['metadata'].get('h1')
    if h1:
        h1_dict[entry['metadata']['page_number']] = h1

    mime_type = entry['metadata'].get('mime_type','text/plain').split('/')
    entry['metadata']['mime_type'] = mime_type[0]
    if len(mime_type) > 1:
        entry['metadata']['mime_subtype'] = '/'.join(mime_type[1:])

    # Identify all header levels in the metadata and sort them
    headers = sorted([key for key in entry['metadata'] if key.startswith('h')], key=lambda x: int(x[1:]))
    if headers:
        # Lowest level header is the last in the sorted list
        lowest_header = headers[-1]
        header_content = entry['metadata'][lowest_header]
        link_header = BidirLinkTag(kind=lowest_header, tag=header_content)
        entry['metadata']['link_tags'] = [link_header]
        entry['metadata']['id'] = sha256(str(entry['page_content']).encode('utf-8')).hexdigest()
    
    doc = Document(page_content=entry['page_content'], metadata=entry['metadata'])
    text_documents.append(doc)


## Images

For images, we will use `PyMuPDF` to extract images from the document, `base64` encode the image, and create a `Document` referencing the appropriate H1 heading for the page. 

In [8]:
import pymupdf
import base64

doc = pymupdf.open('Tourbook.pdf')
image_documents = []

# page_index starts from 0, so these are actual pages 3, 5-23, but are numbered 1 and 3-21. 
pages_to_process = [2] + list(range(4, 23))  

for page_index in pages_to_process:
    page = doc[page_index]
    image_list = page.get_images()
    adjusted_page_number = page_index - 1

    # Iterate over the images on the page
    for image_index, img in enumerate(image_list, start=1):
        xref = img[0]
        pix = pymupdf.Pixmap(doc, xref) 
        if pix.n - pix.alpha > 3:
            pix = pymupdf.Pixmap(pymupdf.csRGB, pix)

        base64_image = base64.b64encode(pix.tobytes(output="png")).decode('utf-8')

        if adjusted_page_number % 2 == 0:  # If it's even
            page_spread = f"{adjusted_page_number}-{adjusted_page_number+1}"
        else:  # If it's odd
            page_spread = f"{adjusted_page_number-1}-{adjusted_page_number}"

        h1 = h1_dict[adjusted_page_number]
        link_h1 = BidirLinkTag(kind="h1", tag=h1)
        doc_metadata = {
            "mime_type": "image",
            "mime_subtype": "png",
            "mime_encoding": "base64",
            "page_number": adjusted_page_number, 
            "page_spread": page_spread, 
            "image_index": image_index,
            "h1": h1, 
            "link_tags" : [ link_h1 ],
            "id": sha256(base64_image.encode('utf-8')).hexdigest()
        }
        # Now in theory, langchain_google_vertex.embeddings.embed_image() calls ImageBytesLoader.load_bytes
        # which can take a base64 string, but that wasn't working...but this URI trick does work!
        image_doc = Document(page_content=f"data:image/png;base64,{base64_image}", metadata=doc_metadata)
        image_documents.append(image_doc)

## Load Knowledge Store

In [10]:
docs = []
ids = []
# GraphStore doesn't currently support metadata filtering, and LangChain doesn't deal well with
# retrieved multimedia. More on this later.
for doc in text_documents: # + image_documents:
    docs.append(doc)
    ids.append(doc.metadata['id'])

    if len(docs) >= 50:
        print("saving batch")
        graph_store.add_documents(docs, ids=ids)
        docs.clear()
        ids.clear()

if docs:
    print("saving batch")
    graph_store.add_documents(docs, ids=ids)

saving batch
saving batch
saving batch
saving batch


# Query Examples

A fairly standard prompt template:

In [15]:
from langchain_core.prompts import ChatPromptTemplate

# The 'without sounding American' part of the prompt is simply trying to tone down the 'enthusiasm',
# resulting in output that is more widely paletable. Feel free to remove it!
template = """
You are a helpful travel agent bot. 
You should provide answers to the travellers questions in a manner that encourages them to travel, without sounding American. 
Include specific details of things to see and do, so that you appear knowledgeable about the destination. Give a detailed itinerary.
Answer the question based STRICTLY on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

Some helper methods:

In [13]:
from IPython.display import display, Markdown

def format_docs(docs):
    formatted = "\n\n".join(f"From {doc.metadata['content_id']}: {doc.page_content}" for doc in docs)
    return formatted

# Helper method to render markdown in responses to a chain.
def run_and_render(chain, question):
    result = chain.invoke(question)
    display(Markdown(result))

And now some queries. Searching the `KnowledgeStore` with a depth of `0` is equivalent to a vector search:

In [20]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# Depth 0 doesn't traverses edges and is equivalent to vector similarity only.
vector_retriever = graph_store.as_retriever(search_kwargs={"depth": 0})

vector_rag_chain = (
    {"context": vector_retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | chatModel
    | StrOutputParser()
)

QUESTION="What can I do in the golden triangle?"
run_and_render(vector_rag_chain, QUESTION)

Ah, the Golden Triangle! A journey through the heart of India, where history whispers in ancient ruins, vibrant colours dance in bustling bazaars, and the Taj Mahal stands as a testament to eternal love. 

You could embark on the **Classic Golden Triangle**, a timeless itinerary that takes you to Delhi, the bustling capital, Agra, home to the Taj Mahal, and Jaipur, the Pink City. 

In Delhi, you'll be swept away by the grandeur of the Red Fort, the intricate beauty of Humayun's Tomb, and the bustling energy of Chandni Chowk.  

Agra will captivate you with the Taj Mahal, a masterpiece of Mughal architecture, and the imposing Agra Fort, a testament to the power of the Mughal emperors. 

Jaipur, the Pink City, will enchant you with its vibrant bazaars, the majestic Hawa Mahal, and the Amber Fort, a hilltop palace with breathtaking views. 

But why stop there? You could **spice up your journey** with an extension to Ranthambore, where you can witness the majestic Bengal tigers in their natural habitat, or to Samode, a charming village with a magnificent fort-palace. 

For a truly immersive experience, consider the **Rustic Charm of the Golden Triangle**, which takes you to Mandawa, a town known for its beautifully painted havelis (traditional mansions). 

No matter which route you choose, the Golden Triangle promises an unforgettable journey through the heart of India. 


One such example output (with `temperature=0`) is:

> Ah, the Golden Triangle! A journey through the heart of India, where history whispers in ancient ruins, vibrant colours dance in bustling bazaars, and the Taj Mahal stands as a testament to eternal love.
> 
> You could embark on the Classic Golden Triangle, a timeless itinerary that takes you to Delhi, the bustling capital, Agra, home to the Taj Mahal, and Jaipur, the Pink City.
> 
> In Delhi, you'll be swept away by the grandeur of the Red Fort, the intricate beauty of Humayun's Tomb, and the bustling energy of Chandni Chowk.
> 
> Agra will captivate you with the Taj Mahal, a masterpiece of Mughal architecture, and the imposing Agra Fort, a testament to the power of the Mughal emperors.
> 
> Jaipur, the Pink City, will enchant you with its vibrant bazaars, the majestic Hawa Mahal, and the Amber Fort, a hilltop palace with breathtaking views.
> 
> But why stop there? You could spice up your journey with an extension to Ranthambore, where you can witness the majestic Bengal tigers in their natural habitat, or to Samode, a charming village with a magnificent fort-palace.
> 
> For a truly immersive experience, consider the Rustic Charm of the Golden Triangle, which takes you to Mandawa, a town known for its beautifully painted havelis (traditional mansions).
> 
> No matter which route you choose, the Golden Triangle promises an unforgettable journey through the heart of India.

Based on documents:

From 29877647e4fefb1a2c6305bfe3f731bdbc628b03ca55c428791ae460fb9fbff5: Suggested Itineraries
- Classic Golden Triangle: Delhi - Agra - Jaipur - Delhi
- Golden Triangle with Tigers: Delhi - Jaipur - Ranthambore Bharatpur - Agra - Delhi
- Golden Triangle with Samode: Delhi - Agra - Samode - Jaipur - Delhi
- Golden Triangle with Neemrana: Delhi - Neemrana - Jaipur - Agra - Delhi
- Rustic Charm of Golden Triangle: Delhi - Mandawa - Jaipur - Agra - Delhi
- Images of North India: Delhi - Jaipur - Agra - Orchha / Khajuraho - Varanasi
- Grand Heritage of North India: Delhi - Jaipur - Agra - Gwalior Orchha / Khajuraho - Varanasi

From 26b4df804db61263d6f80f1654f6dd3b6fae2a531811cd26a5845e52def37516: Golden Triangle Extensions
Spice up the classic Golden Triangle tour by extending it to any the following interesting destinations

From 0fd018f03f0b1f078a2c1d64fc5048115921b22242e8e19f931d04c968af82cd: Golden Triangle
The most popular tour of India, the Golden Triangle tour, includes a visit to Delhi, the capital of India; Agra, land of the world famous Taj Mahal; and Jaipur, the pink capital city of Rajasthan.

From e6f649156034f99f24aec8782914dbf9e3db47881f72a519dbb0cdfd0dc2359e: River Cruises
ASSAM BENGAL NAVIGATION : Assam & Bengal
Experience the hidden beauty and treasures of remote north eastern regions of Assam and Bengal.


Experimenting with different temperartures increases 'hallucinations' or at least not strict adherence to the prompt, but even here we can find references to things not in the context, like "Humayun's Tomb" or "Chandi Chowk". While Chandi Chowk is referenced elsewhere in the travel document, Humayun's Tomb is not (though it is a sight one can see in Delhi).

Let's run the same question, but searching the graph with `depth=1`, which has the effect of searching for paragraphs within the same heading:

In [22]:
# Depth 1 does vector similarity and then traverses 1 level of edges.
graph_retriever = graph_store.as_retriever(search_kwargs={"depth": 1})

graph_rag_chain = (
    {"context": graph_retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | chatModel
    | StrOutputParser()
)
run_and_render(graph_rag_chain, QUESTION)

Retrying langchain_google_vertexai.chat_models._completion_with_retry.<locals>._completion_with_retry_inner in 4.0 seconds as it raised ServiceUnavailable: 503 Connection reset.


Ah, the Golden Triangle! A journey through the heart of India, where history whispers in every breeze.  You will be captivated by the grandeur of Delhi, the Mughal majesty of Agra, and the vibrant colours of Jaipur. 

In Delhi, you'll be swept away by the bustling bazaars, the imposing Red Fort, and the serene beauty of the Lotus Temple.  Agra will unveil the timeless wonder of the Taj Mahal, a monument to love that transcends time.  And Jaipur, the Pink City, will enchant you with its palaces, forts, and the bustling bazaars of Johari Bazaar. 

Here's a suggested itinerary:

**Day 1:** Arrive in Delhi, the capital of India.  Explore the bustling bazaars of Chandni Chowk, the imposing Red Fort, and the serene beauty of the Lotus Temple.

**Day 2:**  Journey to Agra, home to the Taj Mahal.  Witness the sunrise over this iconic monument, a testament to Mughal architecture and love.  In the afternoon, explore the Agra Fort, another architectural marvel.

**Day 3:**  Travel to Jaipur, the Pink City.  Visit the magnificent Amber Fort, a hilltop fortress that offers breathtaking views.  In the evening, stroll through the bustling bazaars of Johari Bazaar, known for its exquisite jewellery.

**Day 4:**  Explore the City Palace, a grand complex that showcases the rich history of Jaipur.  Visit the Hawa Mahal, a palace with 953 windows, designed to allow the royal ladies to observe the city without being seen.

**Day 5:**  Depart from Jaipur, carrying with you the memories of a journey through the Golden Triangle.

This is just a glimpse of what awaits you in the Golden Triangle.  Each city holds a treasure trove of experiences, waiting to be discovered.  So, pack your bags, embrace the adventure, and let the Golden Triangle weave its magic upon you. 


Sample output (`temperature=0`):

> Ah, the Golden Triangle! A journey through the heart of India, where history whispers in every breeze. You will be captivated by the grandeur of Delhi, the Mughal majesty of Agra, and the vibrant colours of Jaipur.
> 
> In Delhi, you'll be swept away by the bustling bazaars, the imposing Red Fort, and the serene beauty of the Lotus Temple. Agra will unveil the timeless wonder of the Taj Mahal, a monument to love that transcends time. And Jaipur, the Pink City, will enchant you with its palaces, forts, and the bustling bazaars of Johari Bazaar.
> 
> Here's a suggested itinerary:
> 
> Day 1: Arrive in Delhi, the capital of India. Explore the bustling bazaars of Chandni Chowk, the imposing Red Fort, and the serene beauty of the Lotus Temple.
> 
> Day 2: Journey to Agra, home to the Taj Mahal. Witness the sunrise over this iconic monument, a testament to Mughal architecture and love. In the afternoon, explore the Agra Fort, another architectural marvel.
> 
> Day 3: Travel to Jaipur, the Pink City. Visit the magnificent Amber Fort, a hilltop fortress that offers breathtaking views. In the evening, stroll through the bustling bazaars of Johari Bazaar, known for its exquisite jewellery.
> 
> Day 4: Explore the City Palace, a grand complex that showcases the rich history of Jaipur. Visit the Hawa Mahal, a palace with 953 windows, designed to allow the royal ladies to observe the city without being seen.
> 
> Day 5: Depart from Jaipur, carrying with you the memories of a journey through the Golden Triangle.
> 
> This is just a glimpse of what awaits you in the Golden Triangle. Each city holds a treasure trove of experiences, waiting to be discovered. So, pack your bags, embrace the adventure, and let the Golden Triangle weave its magic upon you.

Based on documents:

From 29877647e4fefb1a2c6305bfe3f731bdbc628b03ca55c428791ae460fb9fbff5: Suggested Itineraries
- Classic Golden Triangle: Delhi - Agra - Jaipur - Delhi
- Golden Triangle with Tigers: Delhi - Jaipur - Ranthambore Bharatpur - Agra - Delhi
- Golden Triangle with Samode: Delhi - Agra - Samode - Jaipur - Delhi
- Golden Triangle with Neemrana: Delhi - Neemrana - Jaipur - Agra - Delhi
- Rustic Charm of Golden Triangle: Delhi - Mandawa - Jaipur - Agra - Delhi
- Images of North India: Delhi - Jaipur - Agra - Orchha / Khajuraho - Varanasi
- Grand Heritage of North India: Delhi - Jaipur - Agra - Gwalior Orchha / Khajuraho - Varanasi

From 26b4df804db61263d6f80f1654f6dd3b6fae2a531811cd26a5845e52def37516: Golden Triangle Extensions
Spice up the classic Golden Triangle tour by extending it to any the following interesting destinations

From 0fd018f03f0b1f078a2c1d64fc5048115921b22242e8e19f931d04c968af82cd: Golden Triangle
The most popular tour of India, the Golden Triangle tour, includes a visit to Delhi, the capital of India; Agra, land of the world famous Taj Mahal; and Jaipur, the pink capital city of Rajasthan.

From e6f649156034f99f24aec8782914dbf9e3db47881f72a519dbb0cdfd0dc2359e: River Cruises
ASSAM BENGAL NAVIGATION : Assam & Bengal
Experience the hidden beauty and treasures of remote north eastern regions of Assam and Bengal.

From 32dd3042477fed1af9f3de7bd6a2ebde7c25d5bc7342354132a1b67db36f74cd: Suggested Itineraries
- Himalayan Glory: Delhi - Amritsar - Dharamsala - Manali - Keylong - Sarchu - Leh - Alchi - Leh - Delhi
- Himalayan Moonland Tour: Delhi - Manali - Keylong - Sarchu - Leh - Delhi
- Holy Himalayas: Delhi - Haridwar / Rishikesh - Shimla - Manali - Dharamsala - Amritsar - Delhi
- Ladakh & Nubra Valley Jeep Safari: Leh - Alchi - Khardung La Pass - Nubra Valley - Leh
- A Date with Nature: Delhi - Corbett - Rishikesh - Auli - Kaudilya - Haridwar - Delhi

From a5ec2616156d4a504bdb26000bd4e1fb41db4b1115f2a8c807293b10c5af5f5a: Suggested Itineraries
- Mumbai to Kerala Tour: Mumbai - Bangalore - Hassan - Mysore - Ooty - Calicut - Kochi - Periyar - Kumarakom - Kovalam - Trivandrum 
- Jewels of South India: Bangalore - Hassan - Mysore - Ooty - Kochi-Alleppey / Kumarakom - Periyar - Madurai - Trichy / Tanjore-Pondicherry - Mahabalipuram - Kanchipuram / Chennai 
- Deccan Delights: Hyderabad - Gulbarga - Bijapur - Badami - Hampi Goa 
- Temples of South India: Chennai - Mahabalipuram - Pondicherry - Tanjore - Madurai - Kanyakumari - Trivandrum 
- Kerala - God's own country: Kochi - Munnar - Periyar - Kumarakom - Houseboat - Kovalam 
- Emerald Waters of India: Mumbai - Goa - Kochi - Kumarakom - Kovalam

From ee0b7497307e6bc0f5a02526d486ad68de44c01e6a96e72dd8165dab6b5cb3a6: Suggested Itineraries
- Deserts of Rajasthan: Delhi - Mandawa- Bikaner - Jaisalmer - Pokaran - Jodhpur - Udaipur - Jaipur - Agra - Delhi
Forts of Rajasthan: Delhi - Mandawa - Kuchamman - Khimsar - Jaisalmer - Jodhpur - Kumbalgarh - Udaipur - Chittaurgarh -Jaipur - Delhi
Palatial Golden Triangle: Delhi - Agra - Samode - Jaipur - Neemrana - Delhi
Desert Delights: Delhi - Agra - Jaipur - Nimaj - Jodhpur - Pokaran - Jaisalmer - Bikaner - Mandawa - Delhi
Taj Mahal & Gems of Rajasthan: Delhi - Agra - Jaipur - Mandawa Bikaner - Jaisalmer - Jodhpur - Udaipur

From fb7af991f9ab8bda1229af795e54e73475bd52c67f7ba287881bd6716cbfe6a5: Suggested Itineraries
- Highlights of West Bengal and Sikkim : Kolkata - Bagdogra / Darjeeling - Pemayangste - Gangtok - Kalimpong - Bagdogra / Kolkata
- Tribes of Orissa : Bhubaneshwar - Puri - Gopalpur - Rayagada - Jeypore - Gopalpur - Bhubaneshwar
- Assam Valley Tour : Guwahati - Jorhat - Sibasagar - Jorhat - Toklai - Kaziranga - Guwahati
- Natures Paradise : Shillong - Cherrapunji - Kaziranga - Kohima - Touphema - Dimori Cove - Jorhat - Majuli - Guwahati
- Tea Trails : Dibrugarh - Digboi - Jorhat - Kaziranga - Guwahati
- North Eastern Wildlife : Guwahati - Manas - Nameri - Kaziranga Dibrugarh - Dibrusaikhowa - Deban - Dibrugarh

From 2057ac35a9ac14137a9e6d52d4c727a5d3cd1cf5b888c61daa4eab67ad2012da: Cruising through the Holy River Ganges and Hugli, you get an opportunity to see unique cattle fairs of eastern India, holy temples, remains of the British Raj, and some historic Buddhist sites

From 9cced1f0bf264552bb4766042a4f0b31d91a479b0544135060aa6cacdaa2fef2: Enjoy tribal encounters and village walks, interact with the locals, discover tea plantations, explore country towns on cycle rickshaws, relish barbecues on deserted river islands, enjoy folk dance performances, visit temples, shrines and local craft workshops. This is your chance to experience the real rural India... still far from any signs of modernity.

From c720fbb857bc04e3b46b1c07be3488fb9a59594cb2e3cd34d9b482fe314d0c5e: Cruising through River Brahmaputra and Hugli, experience elephant and jeep safaris in some of India's most treasured wildlife parks- Manas, Kaziranga and Orang National Park.

From d9559a68e2fe4c00347435cf9f382b3e94a71174c829860f7fc40a2fb93ec35f: For detailed itineraries, departure dates, and river boat specifications, log on to our website www.traveliteindia.com

In [43]:
mmr_graph_retriever = graph_store.as_retriever(
    search_type = "mmr_traversal",
    search_kwargs = {
        "k": 4,
        "fetch_k": 10,
        "depth": 2,
        # "score_threshold": 0.2,
    },
)

mmr_graph_rag_chain = (
    {"context": mmr_graph_retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | chatModel
    | StrOutputParser()
)

run_and_render(mmr_graph_rag_chain, QUESTION)

Retrying langchain_google_vertexai.chat_models._completion_with_retry.<locals>._completion_with_retry_inner in 4.0 seconds as it raised ServiceUnavailable: 503 Getting metadata from plugin failed with error: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')).


Ah, the Golden Triangle! A journey through the heart of India, where history whispers in ancient ruins, vibrant colours dance in bustling bazaars, and the spirit of India unfolds before your very eyes. 

You can choose from a variety of itineraries, each offering a unique perspective on this captivating region. 

**The Classic Golden Triangle:**  Embark on a timeless journey through Delhi, Agra, and Jaipur. In Delhi, explore the majestic Red Fort, wander through the bustling Chandni Chowk, and delve into the history of the Qutub Minar. In Agra, stand in awe before the Taj Mahal, a testament to eternal love, and marvel at the Agra Fort, a symbol of Mughal power. In Jaipur, the "Pink City," lose yourself in the intricate architecture of the Hawa Mahal, explore the Amber Fort, and experience the vibrant energy of the local markets.

**Golden Triangle with Tigers:**  For the adventurous soul, add a touch of wilderness to your journey. After exploring Delhi, Jaipur, and Agra, venture into Ranthambore National Park, where you can witness the majestic Bengal tiger in its natural habitat.  

**Golden Triangle with Samode or Neemrana:**  For a touch of regal charm, consider adding Samode or Neemrana to your itinerary. These heritage hotels, nestled amidst picturesque landscapes, offer a glimpse into the grandeur of Rajasthan's royal past.

**Rustic Charm of the Golden Triangle:**  For a more authentic experience, explore the hidden gems of Mandawa, a town renowned for its beautifully painted havelis (traditional mansions). 

**Images of North India:**  Extend your journey beyond the Golden Triangle and delve deeper into the cultural tapestry of North India. Visit Orchha, known for its stunning temples, or Khajuraho, famous for its erotic sculptures.  Conclude your journey in Varanasi, the spiritual heart of India, where you can witness the mesmerizing Ganga aarti (prayer ceremony).

**Grand Heritage of North India:**  For a truly immersive experience, embark on a grand tour that encompasses the highlights of North India.  Explore the magnificent Gwalior Fort, a testament to the region's rich history, and immerse yourself in the spiritual aura of Varanasi.

No matter which itinerary you choose, the Golden Triangle promises an unforgettable journey through the heart of India.  


Based on documents:

From 29877647e4fefb1a2c6305bfe3f731bdbc628b03ca55c428791ae460fb9fbff5: Suggested Itineraries
- Classic Golden Triangle: Delhi - Agra - Jaipur - Delhi
- Golden Triangle with Tigers: Delhi - Jaipur - Ranthambore Bharatpur - Agra - Delhi
- Golden Triangle with Samode: Delhi - Agra - Samode - Jaipur - Delhi
- Golden Triangle with Neemrana: Delhi - Neemrana - Jaipur - Agra - Delhi
- Rustic Charm of Golden Triangle: Delhi - Mandawa - Jaipur - Agra - Delhi
- Images of North India: Delhi - Jaipur - Agra - Orchha / Khajuraho - Varanasi
- Grand Heritage of North India: Delhi - Jaipur - Agra - Gwalior Orchha / Khajuraho - Varanasi

From 26b4df804db61263d6f80f1654f6dd3b6fae2a531811cd26a5845e52def37516: Golden Triangle Extensions
Spice up the classic Golden Triangle tour by extending it to any the following interesting destinations

From d820c47547490681c04ababf7d66397bcd570c62fd5c9dfd280ab16f990c3d19: When travelling to India, it is worth taking a short trip to a neighbouring country, to enhance your experience. You will notice a striking similarity, yet subtle differences in the people, food, culture, religion and lifestyle in the entire subcontinent. It is in these differences that we celebrate the 'unity in diversity of the Indian subcontinent.

From 14345787c02a3d3eedf901803acce8ba4d1e6ef86470355884d8149129b467c5: LUMBINI (Nepal), the celebrated birthplace of Lord Buddha.

No idea what it's done here on the MMR retriever, but it's very little  to work from so it's filled in a lot of detail from outside the context...

In [42]:
import cassio.config
questionVec = embedModel.embed([QUESTION])
cc = cassio.config.resolve_session()

k4 = ['29877647e4fefb1a2c6305bfe3f731bdbc628b03ca55c428791ae460fb9fbff5',
      '26b4df804db61263d6f80f1654f6dd3b6fae2a531811cd26a5845e52def37516',
      '0fd018f03f0b1f078a2c1d64fc5048115921b22242e8e19f931d04c968af82cd',
      'e6f649156034f99f24aec8782914dbf9e3db47881f72a519dbb0cdfd0dc2359e']

query = "SELECT similarity_cosine(text_embedding,?) as sim FROM default_keyspace.travel_docs_nodes WHERE content_id = ?"
pstmt = cc.prepare(query)

for k in k4:
    bound_stmt = pstmt.bind([questionVec[0], k])
    rows = cc.execute(bound_stmt)
    row = rows.one()
    print(f"Sim for {k}: {row.sim}")

Sim for 29877647e4fefb1a2c6305bfe3f731bdbc628b03ca55c428791ae460fb9fbff5: 0.7930660247802734
Sim for 26b4df804db61263d6f80f1654f6dd3b6fae2a531811cd26a5845e52def37516: 0.7651149034500122
Sim for 0fd018f03f0b1f078a2c1d64fc5048115921b22242e8e19f931d04c968af82cd: 0.7594711184501648
Sim for e6f649156034f99f24aec8782914dbf9e3db47881f72a519dbb0cdfd0dc2359e: 0.7589305639266968


This image is not in the brochure:

![](camel-riding.jpg)!

In [45]:
from langchain_core.messages import HumanMessage

image_message = {
    "type": "image_url",
    "image_url": {"url":"camel-riding.jpg"},
}

text_message = {
    "type": "text",
    "text": "Where can I do this while on my trip around the golden triangle?",
}
message = HumanMessage(content=[text_message, image_message])

output = chatModel.invoke([message])
display(Markdown(output.content))


Retrying langchain_google_vertexai.chat_models._completion_with_retry.<locals>._completion_with_retry_inner in 4.0 seconds as it raised ServiceUnavailable: 503 Connection reset.


You can go on a camel safari in the Thar Desert near Jaisalmer, Rajasthan. Jaisalmer is a popular destination within the Golden Triangle. 


In [46]:
from langchain_core.messages import HumanMessage
from langchain_core.messages import SystemMessage
from langchain_core.language_models import BaseChatModel

def describe_image(llm: BaseChatModel, image_url: str, instruct: str) -> str:
    """Describe the contents of an image."""
    system_message = SystemMessage(content="""
        You are a tool that converts images to text.     
    """)

    image_message = HumanMessage(content=[
    {"type": "text", "text": instruct or "Describe the contents of this image."},
    {"type": "image_url","image_url": {"url":image_url}}
    ])
    
    output = llm.invoke([system_message, image_message])
    return output.content
    
image_description = describe_image(chatModel,"camel-riding.jpg","Describe the image as completely as possible in 50-80 words, in a manner that is suitable for searching travel brochures.")
IMAGE_QUESTION=f"Image: {image_description}\n Where can I do this while on my trip around the golden triangle?"
run_and_render(graph_rag_chain, IMAGE_QUESTION)

Ah, the Thar Desert! A truly magnificent landscape, and a camel safari is the perfect way to experience its beauty.  You can embark on this adventure in Jaisalmer, the "Golden City" of Rajasthan.  

Jaisalmer is a part of the Golden Triangle, a popular tourist circuit that also includes Delhi and Agra.  

Here's a possible itinerary for your trip:

**Day 1:** Arrive in Delhi, the bustling capital of India.  Explore the city's historical monuments, such as the Red Fort, Jama Masjid, and Qutub Minar.  In the evening, enjoy a delicious dinner at one of Delhi's many street food stalls.

**Day 2:** Take a morning flight to Jaisalmer.  Check into your desert camp, perhaps the Dune Safari Camp, known for its rustic charm and its ability to arrange desert marathons and survival lessons.  After settling in, embark on your camel safari.  As you ride through the dunes, you'll be surrounded by the breathtaking beauty of the desert, and you'll have the chance to learn about the unique culture of the Rajasthani people.  In the evening, enjoy a traditional Rajasthani dinner under the stars, accompanied by music and dance.

**Day 3:**  Return to Jaisalmer and explore the city's magnificent Jaisalmer Fort, the second oldest fort in Rajasthan.  Wander through the fort's narrow streets and admire the intricate architecture of the havelis, the grand mansions of the Rajput Maharajas.  

**Day 4:**  Fly from Jaisalmer to Agra, home to the Taj Mahal.  Spend the afternoon exploring this iconic monument, a testament to the love of Mughal Emperor Shah Jahan for his beloved wife Mumtaz Mahal.  

**Day 5:**  Return to Delhi, and enjoy some last-minute shopping for souvenirs before departing for your onward journey.

This itinerary is just a suggestion, of course.  You can customize it to fit your interests and budget.  But I highly recommend including a camel safari in Jaisalmer.  It's an experience you won't soon forget! 


From documents:

From 8df175fc1cc741e51c545d8d12e62ace4787f01747796d8505432c4f45573fe7: JAISALMER, the golden city of Rajasthan and a haven for desert camps offers a wonderful selection of camps to suit every budget. Popular desert camps in and around Jaisalmer include: 
- The Serai, a luxury desert camp and spa, offers 21 tents, including a 'Royal Suite' with its private pool, spa and lounging tent and 6 'Luxury Tents' with private walled gardens and plunge pools. 
- Mirvana Nature Resort, located midway between Pokhran and Jaisalmer, provides 15 luxury tents, raised on a marble platform, crafted using local natural materials. Being a nature resort, Mirvana also offers organic food and fresh farm dairy. 
- Mool Sagar Camp, a historic camp with 18 luxurious tents, delightful stone pavilions and fountains, and a swimming pool, set amidst wonderful gardens. 
- Dune Safari Camp, in the sand dunes of Sam, consists of 20 deluxe tents, with a rustic charm. The camp can also arrange desert marathons and desert survival lessons for small groups.

From 9e8aad6fed1134dc8e97748c32f61b58f3e27ce4fcef4707f4b7ad4573dbe47d: Desert Camps of Rajasthan
Rajasthan, home to the golden sand dunes, is a haven for desert camps in India. For the erstwhile Maharajas, campaigning in distant lands meant long stays in desert camps. During wars or expeditions, camps were the only luxury available and soon these “homes away from home” evolved into mobile, mini-palaces.
Today, Rajasthan offers a wonderful selection of desert camps, to create the ideal nomadic desert retreat. Most camps offer camel and jeep safaris to nearby villages and evening entertainment with bon fire, local musicians & dancers, puppet shows, juggling, and other regional performances.

From e5a09cfb00e1d277177bc5ab01209a85256d0095c119263e0cf7f045df2fffc4: Allow us to create an experience of a lifetime, an inspiring journey to an exotic land, discovering the people, culture, tradition, history and legacy of our Incredible India and its neighbouring wonderlands. With our passion, experience and meticulous planning, we promise to weave memories you will cherish forever.

From df069b310d95239bb18399fdd6466500f3631a2914dc501efa37f704e7fbf384: BIKANER, the camel city of India, is renowned for the best riding camels in the world. The Junagarh Fort in Bikaner is an impressive formidable structure built in the 16th century. The relatively modern red sandstone palace, Lalgarh Palace, was built in the early 20th century and is an architectural marvel surrounded by sprawling lawns and gardens.

From e86b0c06c110ec5a997ad26cad68efb197e0d4016d81a08b9ee03b13c21d1634: JAISALMER, the city of golden sand dunes, in the far west of Rajasthan, is popularly called 'the golden city of India . Jaisalmer Fort is the second oldest fort of Rajasthan, encapsulating five beautiful palaces. The grand havelis of Jaisalmer, featuring intricate designs and luxurious interiors, are fine examples of architectural brilliance of the Rajput Maharajas.

From 1d97d3c07ddd009f2bc1a4d5f882b9785e9d2e77c6d964b754f3e22400b1841c: Welcome aboard your Indian voyage with Travelite (India)

From 51f57a627e579114432573624699c8011c7e81f439b50875e7b253e896f19cd9: India is a medley of fascinating colours and cultures, an historical legacy, a canvas of architectural masterpieces, and an extravagant exuberance of royal splendour. This land of sages and ancient legends is home to the eternal symbol of love - the Taj Mahal, magnificent forts and palaces of Rajasthan, emerald beaches of Goa and Kerala, ancient caves of Ajanta and Ellora and the Golden Temple of Amritsar.

From 795bc833b145bf2bb7bb409e7fd680ebeb8ac3498739b513274aa8af686f9466: In this brochure, we introduce you to the mesmerising beauty of the Indian subcontinent. Combining the cultural splendours of India with the snow covered Himalayan peaks of Nepal, the serenity of Bhutan, green tea estates of Sri Lanka and the turquoise beaches of Maldives, the Indian sub continent treasures it all.

From c07d382d6ef0372001e9deeff943efe7fd73307d67e5e5f0e40dbab7c2ce4e7f: Crowned by Himalayas in the north and surrounded by oceans in the south, India offers a platter full of variety, packed in one country. Ranging from walking trails through Himalayan villages, to desert camps in the sand dunes, tribal and rural experiences, modern metropolitan cities, wildlife parks, serene backwaters, luxury trains, river cruises and adventure tours, the choice is only limited by your imagination!

From d820c47547490681c04ababf7d66397bcd570c62fd5c9dfd280ab16f990c3d19: When travelling to India, it is worth taking a short trip to a neighbouring country, to enhance your experience. You will notice a striking similarity, yet subtle differences in the people, food, culture, religion and lifestyle in the entire subcontinent. It is in these differences that we celebrate the 'unity in diversity of the Indian subcontinent.